In [27]:
import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms

# 미리 작성된 코드들은 수정할 수 없으며, 이외의 코드를 작성하시면 됩니다.

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)


class Classifier(nn.Module):
    # 모델의 코드는 여기서 작성해주세요
    

    def __init__(self):
        super(Classifier, self).__init__()
        self.linear1=nn.Linear(32*32*3,32*32)
        self.linear2=nn.Linear(32*32,512)
        self.linear3=nn.Linear(512,256)
        self.linear4=nn.Linear(256,10)
        
        self.bn1=nn.BatchNorm1d(32*32)
        self.bn2=nn.BatchNorm1d(512)
        self.bn3=nn.BatchNorm1d(256)
        self.activation=nn.ReLU()
        self.acti2=nn.Sigmoid()
        self.dropout=nn.Dropout(0.35)

    def forward(self, x):
        z1=self.linear1(x)
        z1=self.bn1(z1)
        a1=self.activation(z1)
        a1=self.dropout(a1)

        z2=self.linear2(a1)
        z2=self.bn2(z2)
        a2=self.activation(z2)
        a2=self.dropout(a2)

        z3=self.linear3(a2)
        z3=self.bn3(z3)
        a3=self.activation(z3)

        output=self.linear4(a3)
        return output


if __name__ == "__main__":
    # 학습코드는 모두 여기서 작성해주세요
    transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.4914,0.4822,0.4465),(0.247,0.243,0.261))])

    train_dataset = torchvision.datasets.CIFAR10(root="CIFAR10/",
                                                 train=True,
                                                 transform=transform,
                                                 download=True)
    test_dataset = torchvision.datasets.CIFAR10(root="CIFAR10/",
                                                train=False,
                                                transform=transform,
                                                download=True)
    


    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = Classifier()
    model=model.to(device)
    model.train()
    optimizer=torch.optim.Adam(params=model.parameters(),lr=0.001,betas=(0.9,0.999))
    criterion=nn.CrossEntropyLoss()



    batch_size=64
    train_dataloader=torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
    total_batch_num=len(train_dataloader)
    epochs=29  
    for epoch in range(epochs):
        avg_cost=0
        for b_x,b_y in train_dataloader:
          b_x=b_x.view(-1,32*32*3).to(device)
          logits=model(b_x)
          loss=criterion(logits , b_y.to(device))
          
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

          avg_cost+=loss/total_batch_num
        print("Epoch : {} / {} , Cost : {}".format(epoch+1,epochs,avg_cost))

    #===================================
    #            train code 
    #===================================

    torch.save(model.state_dict(), 'model.pt')  # 학습된 모델을 저장하는 코드입니다.


Files already downloaded and verified
Files already downloaded and verified
Epoch : 1 / 29 , Cost : 1.6871795654296875
Epoch : 2 / 29 , Cost : 1.5033833980560303
Epoch : 3 / 29 , Cost : 1.426282525062561
Epoch : 4 / 29 , Cost : 1.3678505420684814
Epoch : 5 / 29 , Cost : 1.3192986249923706
Epoch : 6 / 29 , Cost : 1.2769315242767334
Epoch : 7 / 29 , Cost : 1.2400115728378296
Epoch : 8 / 29 , Cost : 1.2078399658203125
Epoch : 9 / 29 , Cost : 1.1709697246551514
Epoch : 10 / 29 , Cost : 1.1397747993469238
Epoch : 11 / 29 , Cost : 1.1113383769989014
Epoch : 12 / 29 , Cost : 1.0836713314056396
Epoch : 13 / 29 , Cost : 1.060659408569336
Epoch : 14 / 29 , Cost : 1.0319570302963257
Epoch : 15 / 29 , Cost : 1.009769082069397
Epoch : 16 / 29 , Cost : 0.9858441352844238
Epoch : 17 / 29 , Cost : 0.9576980471611023
Epoch : 18 / 29 , Cost : 0.9369521737098694
Epoch : 19 / 29 , Cost : 0.9144992828369141
Epoch : 20 / 29 , Cost : 0.890821635723114
Epoch : 21 / 29 , Cost : 0.8701067566871643
Epoch : 22 / 

In [28]:
# 학습된 모델의 성능을 평가하는 코드입니다.
# 아래의 코드로 평가를 진행할 예정이므로 아래의 코드가 정상 동작 해야하며, 제출전 모델의 성능을 확인하시면 됩니다.

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

test_dataset = torchvision.datasets.CIFAR10(root="CIFAR10/",
                                            train=False,
                                            transform=transform,
                                            download=True)


test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=10000)

classifier = Classifier().to(device)
classifier.load_state_dict(torch.load('model.pt'))
classifier.eval()


for data, label in test_dataloader:
    data = data.view(-1, 32 * 32 * 3).to(device)

    with torch.no_grad():
        logits = classifier(data)

        pred = torch.argmax(logits, dim=1)

        total = len(label)
        correct = torch.eq(pred, label.to(device)).sum()

        print("Accuracy on test set : {:.4f}%".format(100 * correct / total))

Files already downloaded and verified
Accuracy on test set : 59.3500%
